In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def request_load(_id):
    session = requests.Session()
    timeout = 10
    headers = {
        'authority': 'txdot.public.ms2soft.com',
        'accept': '*/*',
        'accept-language': 'en,zh;q=0.9,zh-CN;q=0.8,ja;q=0.7,zh-TW;q=0.6',
        'referer': 'https://txdot.public.ms2soft.com/tcds/tdetail.asp?updatemap=&from_map=',
        'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    }
    res = session.get('https://txdot.public.ms2soft.com/tcds/tsearch.asp', headers = headers) #request for home page and store the cookies in cks
    cks = session.cookies

    lst = [] #create an empty list to append all responses
    
    for pg in range(1,6):  #AADT detail has at most 5 pages
        params = {
            'offset': '0',
            'agency_id': '97',
            'local_id': _id,
            'page_type': '',
            'pg': str(pg),
        }

        response = requests.get(
            'https://txdot.public.ms2soft.com/tcds/ajax/tcds_tdetail_aadt.asp',
            params=params,
            timeout = timeout,
            headers=headers,
            cookies = cks
        )
        
        lst.append(response.text)
        
    return lst
    

    

In [3]:
lst = request_load('15H193')

In [ ]:
for response in lst:
    print(response)

In [13]:
col_nm = []
row_data = []

for response in lst:
    soup = BeautifulSoup(response, 'html.parser')
    rows_tr = soup.find_all('tr', class_ = 'FormRowLabel')

    # Getting column names
    # if col_nm:
    #     pass
    # else:
    #     for cell in rows_tr[0]:
    #         if cell.text.strip() != '':
    #             col_nm.append(cell.text.strip())
    if not col_nm:
        col_nm = [cell.text.strip() for cell in rows_tr[0] if cell.text.strip()]
    
    # Getting values by rows
    for i in range(1, len(rows_tr)):
        row = rows_tr[i].find_all('td')
        if len(row) <= 1:
            break
        cell_data = [cell_bs.text for cell_bs in row if cell_bs.text != '']
        row_data.append(cell_data)

    # for i in range(len(rows_tr)-1):
    #     row = rows_tr[i+1].find_all('td')
    #     if len(row) <= 1:
    #         break
    #     data = []
    #     for cell_bs in row:
    #         cell = cell_bs.text
    #         if cell != '':
    #             data.append(cell)
    #     row_data.append(data)   


In [14]:
row_data

[['2022', '14,370', '1,319', '9', '59', '13,495 (94%)', '875 (6%)', '\xa0'],
 ['2021', '13,014', '1,058', '8', '51', '12,176 (94%)', '838 (6%)', '\xa0'],
 ['2020', '10,385', '1,217', '12', '56', '9,622 (93%)', '763 (7%)', '\xa0'],
 ['2019', '11,731', '1,292', '11', '55', '10,930 (93%)', '801 (7%)', '\xa0'],
 ['2018', '10,115', '1,296', '13', '53', '9,191 (91%)', '924 (9%)', '\xa0'],
 ['2017', '10,974', '\xa0', '\xa0', '\xa0', '9,987 (91%)', '987 (9%)', '\xa0'],
 ['2016', '10,741', '1,274', '12', '54', '9,686 (90%)', '1,055 (10%)', '\xa0'],
 ['2015', '11,622', '1,262', '11', '55', '10,872 (94%)', '750 (6%)', '\xa0'],
 ['2014', '9,684', '979', '8', '51', '8,834 (91%)', '829 (9%)', '\xa0'],
 ['2013', '11,308', '1,239', '11', '54', '\xa0', '\xa0', '\xa0'],
 ['2012', '10,200', '1,457', '8', '51', '9,395 (92%)', '805 (8%)', '\xa0'],
 ['2011', '10,000', '1,428', '8', '51', '9,190 (92%)', '810 (8%)', '\xa0'],
 ['2010', '9,900', '\xa0', '\xa0', '\xa0', '\xa0', '\xa0', '\xa0'],
 ['2009', '10,700

In [15]:
aadt = pd.DataFrame(row_data, columns = col_nm)

In [36]:
aadt

,Year,AADT,DHV-30,K %,D %,PA,BC,Src
0,2022,"14,370","1,319",9,59,"13,495 (94%)",875 (6%),
1,2021,"13,014","1,058",8,51,"12,176 (94%)",838 (6%),
2,2020,"10,385","1,217",12,56,"9,622 (93%)",763 (7%),
3,2019,"11,731","1,292",11,55,"10,930 (93%)",801 (7%),
4,2018,"10,115","1,296",13,53,"9,191 (91%)",924 (9%),
5,2017,"10,974",,,,"9,987 (91%)",987 (9%),
6,2016,"10,741","1,274",12,54,"9,686 (90%)","1,055 (10%)",
7,2015,"11,622","1,262",11,55,"10,872 (94%)",750 (6%),
8,2014,"9,684",979,8,51,"8,834 (91%)",829 (9%),
9,2013,"11,308","1,239",11,54,,,


In [18]:
aadt.to_csv('historical_aadt_15H193.csv', index=False)  
print(f'Data saved to "traffic_data.csv"')

Data saved to "traffic_data.csv"
